<a href="https://colab.research.google.com/github/vnavya2004/Depressiondetection_BTP/blob/main/GraphAutoEncoder_Example_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
"""
Number of data points: 300
Number of classes: 20
"""

# INPUTS
W = np.ones((300, 200))
X = np.ones((300, 200))

# OUTPUT
Y = np.random.rand(300, 20)

In [ ]:
# Train test split
W_train, X_train, Y_train = W[:200], X[:200], Y[:200]
W_test, X_test, Y_test = W[200:], X[200:], Y[200:]

In [ ]:
!pip install spectral

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install spektral

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 1.7 MB/s eta 0:00:00


In [ ]:
from spektral.layers import GraphSageConv
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, Input, Concatenate
from tensorflow.keras.models import Model

In [ ]:
batch_size = 32
graph_embedding_size = 300

In [ ]:
"""
GAE (Graph Auto Encoder) General Idea

GAE = Encoder + Decoder
Encoder: GCN layer (Can use more than 1 GCN layers)
Decoder: Function which reconstructs the adjacency matrix

Initial_Node_Embedding (N) + Adjacency_Matrix (A) -> Encoder -> Updated_Node_Embedding (N')
Updated_Node_Embedding (N') -> Decoder -> Reconstructed_Adjacency_Matrix (A')

Objective: A and A' should be identical

In the given example:
Encoder: GraphSAGE
Decoder: sigmoid(N' * transpose(N'))
Loss: |A-A'|

For classification based task, we will have 2 loss functions:
1. Categorical Cross Entropy (Classification)
2. Norm (GAE)

"""

# GAE Loss Function
def custom_gae_loss(A_true, A_pred):
    return tf.norm(A_true - A_pred)


def myModel():
    inputs_W = Input(shape=(200), name='W') # [32 X 200]
    inputs_X = Input(shape=(200), name='X') # [32 X 200]

    inputs_sparse = Input(shape=(2*batch_size,), name='sparse_matrix', sparse=True, dtype=tf.float64)  # [64 X 64]

    # GraphSage
    inpEmbedding = Concatenate(axis=0)([inputs_W, inputs_X]) # [64 X 200]
    outEmbedding = GraphSageConv(channels=graph_embedding_size)([inpEmbedding, inputs_sparse]) # [64 X 300]

    # Reconstructed Adjacency Matrix
    A_reconstructed =  tf.linalg.matmul(outEmbedding,tf.transpose(outEmbedding)) # [64 X 64]
    A_reconstructed = tf.math.sigmoid(A_reconstructed) # [64 X 64]

    W_modified = outEmbedding[0:batch_size] # [32 X 300]
    X_modified = outEmbedding[batch_size:2*batch_size] # [32 X 300]

    # Final Layers
    f = Concatenate(axis=1)([W_modified, X_modified]) # [32 X 600]
    dropout = Dropout(0.5)(f)
    Softmax = Dense(20, activation="softmax")(dropout)
    model = Model(inputs=[
        inputs_W,
        inputs_X,
        inputs_sparse,
    ], outputs=[Softmax, A_reconstructed])

    model.compile(optimizer='adam', loss=['categorical_crossentropy', custom_gae_loss])

    return model

model = myModel()

print("Inputs:",[k.shape for k in model.inputs])
print("Outputs:", [k.shape for k in model.outputs], "\n")

print(model.summary())

del model

Inputs: [TensorShape([None, 200]), TensorShape([None, 200]), TensorShape([None, 64])]
Outputs: [TensorShape([None, 20]), TensorShape([None, None])] 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 W (InputLayer)              [(None, 200)]                0         []                            
                                                                                                  
 X (InputLayer)              [(None, 200)]                0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 200)                  0         ['W[0][0]',                   
                                                                     'X[0][0]']                   
                                           

In [ ]:
def cos_sim(A, B):
    return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

def getIndicesAndValues(batch_W, batch_X):
    indices = []
    values = []

    """
    In the constructed graph

    [0:batch_size] => W
    [batch_size:] => X

    Ex:
    Batch Size: 32

    0: Img1
    1: Img2
    .
    .

    31: Img32
    32: Text1
    33: Text2
    .
    .
    63: Text32

    """

    # W <-> W edges
    for i in range(batch_size):
        for j in range(batch_size):
            # Some Criterion
            sim_score = cos_sim(batch_W[i],batch_W[j])
            if sim_score >= 0.5:
                indices.append([i,j])
                values.append(sim_score)

    # X <-> X edges
    for i in range(batch_size):
        for j in range(batch_size):
            # Some Criterion
            sim_score = cos_sim(batch_X[i],batch_X[j])
            if sim_score >= 0.5:
                indices.append([i+batch_size,j+batch_size])
                values.append(sim_score)

    """
    # For W <-> X edges, indices would be of the form [i, j+batch_size]
    for i in range(batch_size):
        for j in range(batch_size):
            # Some Criterion
            if criterion(batch_X[i],batch_X[j]) is True:
                indices.append([i,j+batch_size])
                values.append(edge_weight)
    """

    indices = np.array(indices)
    values = np.array(values)

    return indices, values


def getMatrix(indices, values, dim):
    a = np.zeros((dim,dim))
    for c,(i,j) in enumerate(indices):
        a[i][j] = values[c]
    return a

In [ ]:
def datagenerator(W, X, Y):
    i = 0
    while True:
        if i + batch_size <= len(W):

            inp_W = W[i:i+batch_size]
            inp_X = X[i:i+batch_size]

            # Get Indices and Values
            indices, values = getIndicesAndValues(inp_W, inp_X)

            # Construct Sparse Matrix for GraphSAGE
            inp_sparse = tf.SparseTensor(
                            indices = indices,
                            values = values,
                            dense_shape = (batch_size*2,batch_size*2)
                        )

            out_Y = Y[i:i+batch_size]

            # Construct "Normal" Adjacency Matrix (for GAE Loss function)
            A = getMatrix(indices, values, batch_size*2)

            yield [inp_W, inp_X, inp_sparse], [out_Y, A]
            i+=batch_size
        else:
            i= 0
            continue

x = next(datagenerator(W, X, Y))

print("Inputs:")
for a in x[0]:
    print(a.shape)

print("\nOutputs:")
for a in x[1]:
    print(a.shape)

del x

Inputs:
(32, 200)
(32, 200)
(64, 64)

Outputs:
(32, 20)
(64, 64)


In [ ]:
model = myModel()

num_epochs = 5

for e in range(num_epochs):
    print("Epoch:",e+1)

    history = model.fit(x=datagenerator(W_train, X_train, Y_train),
                        steps_per_epoch=int(len(W_train) / batch_size),
                        epochs=1,
                        verbose=1
                        )

    y_pred = model.predict(x = datagenerator(W_test, X_test, Y_test),
                          steps=int(len(W_test) / batch_size)
                          )

    # Code For evaluation....

Epoch: 1
3/3 [==============================] - 0s 35ms/step
Epoch: 2
3/3 [==============================] - 0s 33ms/step
Epoch: 3
3/3 [==============================] - 0s 33ms/step
Epoch: 4
3/3 [==============================] - 0s 58ms/step
Epoch: 5
3/3 [==============================] - 0s 61ms/step
